In [1]:
# ********************************************************************************
# IMPORTANT: BEFORE RUNNING THIS CELL, PERFORM A "FACTORY RESET RUNTIME" (Colab)
# OR THE EQUIVALENT DEEPEST RESTART IN YOUR ENVIRONMENT (e.g., Kaggle Session Restart).
# THEN, RUN THIS CELL AS THE VERY FIRST CODE IN YOUR NOTEBOOK.
# ********************************************************************************
!pip install -U scikit-learn==1.3.2 imbalanced-learn==0.12.3
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from collections import Counter
import ast
import geopandas as gpd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

import gc


# Now, the imports should work if the environment is truly clean
try:
    from imblearn.over_sampling import SMOTE
    print("\nSuccessfully imported SMOTE.")
except ImportError as e:
    print(f"\nCRITICAL ERROR: Failed to import SMOTE even after aggressive reinstallation: {e}")
    print("This indicates a severe, persistent environment issue.")
    print("Please double-check that you performed a 'Factory reset runtime' (Colab) or equivalent.")
    exit()

# 1. Inspect and Load GeoJSON Files (Modified for Zero Imputation)
data_dir = "/kaggle/input/mar-oct"  # Replace with your folder path
all_features = []
all_labels = []
invalid_samples = []
invalid_bands = Counter()
species_counts = Counter()

# Updated bands list to include all relevant bands
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'NDVI', 'EVI', 'SAVI', 'NDWI', 'DEM']
months = ['', '_1', '_2', '_3', '_4', '_5', '_6', '_7']
band_columns = [band + month for month in months for band in bands]

# Inspect first file
first_file = os.path.join(data_dir, os.listdir(data_dir)[0]) if os.listdir(data_dir) else None
if first_file and first_file.endswith(".geojson"):
    gdf = gpd.read_file(first_file)
    print("Inspecting first 2 rows of first GeoJSON file:")
    for idx in range(min(2, len(gdf))):
        print(f"\nRow {idx}:")
        for band in ['B1', 'B2', 'B11', 'NDVI', 'DEM', 'B2_1', 'NDVI_7']:
            if band in gdf.columns:
                data = gdf[band].iloc[idx]
                try:
                    parsed_data = ast.literal_eval(data) if isinstance(data, str) else data
                    array = np.array(parsed_data, dtype=np.float32)
                    print(f"  Band {band}: shape={array.shape}, first few values={array.flatten()[:5]}")
                except (ValueError, SyntaxError, TypeError) as e:
                    print(f"  Band {band}: Error parsing/converting: {e}")
            else:
                print(f"  Band {band}: Not found in GeoJSON file")

# Load all GeoJSON files
total_samples_attempted = 0
for file in os.listdir(data_dir):
    if file.endswith(".geojson"):
        try:
            gdf = gpd.read_file(os.path.join(data_dir, file))
            print(f"Processing file: {file}, Rows: {len(gdf)}")
            total_samples_attempted += len(gdf)
            for idx, row in gdf.iterrows():
                try:
                    patch = []
                    for col in band_columns:
                        if col not in gdf.columns:
                            invalid_bands[col] += 1
                            array = np.zeros((5, 5), dtype=np.float32)  # Impute with zeros
                            patch.append(array)
                            continue
                        data = row[col]
                        if data is None or (isinstance(data, str) and data.lower() == 'none'):
                            invalid_bands[col] += 1
                            array = np.zeros((5, 5), dtype=np.float32)  # Impute with zeros
                            patch.append(array)
                            continue
                        try:
                            parsed_data = ast.literal_eval(data) if isinstance(data, str) else data
                            array = np.array(parsed_data, dtype=np.float32).reshape(5, 5)
                        except (ValueError, SyntaxError, TypeError) as e:
                            invalid_bands[col] += 1
                            array = np.zeros((5, 5), dtype=np.float32)  # Impute for parsing errors
                            patch.append(array)
                            continue
                        patch.append(array)
                    patch = np.stack(patch, axis=-1)
                    if patch.shape != (5, 5, 136):  # Expected shape: 17 bands * 8 months
                        raise ValueError(f"Unexpected patch shape: {patch.shape}")
                    all_features.append(patch)
                    all_labels.append(row['l3_species'])
                    species_counts[row['l3_species']] += 1
                except (ValueError, SyntaxError, TypeError) as e:
                    invalid_samples.append((file, idx, str(e)))
                    continue
        except Exception as e:
            print(f"Failed to process file {file}: {e}")
            continue

# Log invalid samples and bands
print(f"\nTotal samples attempted: {total_samples_attempted}")
print(f"Valid samples processed: {len(all_features)}")
if invalid_samples:
    print(f"\nSkipped {len(invalid_samples)} invalid samples:")
    for file, idx, error in invalid_samples:
        print(f"File: {file}, Row: {idx}, Error: {error}")
if invalid_bands:
    print("\nBands with None or missing values:")
    for band, count in invalid_bands.most_common():
        print(f"  {band}: {count} times")
print("\nValid samples per species:")
for species, count in species_counts.most_common():
    print(f"  {species}: {count}")

# Convert to NumPy arrays
if not all_features:
    print("\nError: No valid samples loaded. Using Random Forest with dummy data.")
    X_dummy = np.random.rand(100, 5*5*136)  # Updated for 136 channels
    y_dummy = np.random.randint(0, 5, 100)
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_dummy, y_dummy)
    print("Random Forest dummy accuracy:", rf.score(X_dummy, y_dummy))
    print("Please re-export data with updated GEE code.")
    exit()

X = np.array(all_features, dtype=np.float32)  # Shape: (N, 5, 5, 136)
y = np.array(all_labels)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.13.0
    Uninstalling imbalanced-learn-0.13.0:
      Successfully uninstalled imbalanced-learn-0.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.


2025-07-17 15:40:05.220981: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752766805.433761      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752766805.496620      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



Successfully imported SMOTE.
Inspecting first 2 rows of first GeoJSON file:

Row 0:
  Band B1: shape=(5, 5), first few values=[0.1118 0.1158 0.1158 0.1158 0.1158]
  Band B2: shape=(5, 5), first few values=[0.11225 0.11335 0.11335 0.11955 0.11955]
  Band B11: shape=(5, 5), first few values=[0.17015 0.1891  0.1891  0.1891  0.1891 ]
  Band NDVI: shape=(5, 5), first few values=[0.27416557 0.2273243  0.2273243  0.28395182 0.28395182]
  Band DEM: shape=(5, 5), first few values=[88. 88. 88. 88. 88.]
  Band B2_1: shape=(), first few values=[nan]
  Band NDVI_7: shape=(), first few values=[nan]

Row 1:
  Band B1: shape=(5, 5), first few values=[0.125 0.124 0.124 0.124 0.124]
  Band B2: shape=(5, 5), first few values=[0.1239 0.1176 0.1176 0.1152 0.117 ]
  Band B11: shape=(5, 5), first few values=[0.1976 0.1797 0.1797 0.1797 0.1779]
  Band NDVI: shape=(5, 5), first few values=[0.40535372 0.36756483 0.36756483 0.36180538 0.34991294]
  Band DEM: shape=(5, 5), first few values=[94. 94. 93. 93. 93.]


In [2]:
import numpy as np
import joblib
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE

# 1. Encode class labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 2. Flatten features
X_flat = X.reshape(X.shape[0], -1)

# 3. Initial train/temp split
X_train_raw, X_temp, y_train_raw, y_temp = train_test_split(
    X_flat, y_encoded, test_size=0.3, stratify=y_encoded, random_state=42
)

# 4. Apply SMOTE only to training set
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_raw, y_train_raw)

# 5. Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_temp_scaled = scaler.transform(X_temp)

# 6. Split temp into val and test sets
X_val, X_test, y_val, y_test = train_test_split(
    X_temp_scaled, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

# 7. Compute class weights from the resampled training data
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_resampled), y=y_train_resampled)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# 8. Hyperparameter tuning for Random Forest
param_dist = {
    'n_estimators': [100, 200],
    'max_depth': [None, 20, 40],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

rf = RandomForestClassifier(random_state=42, class_weight=class_weight_dict, n_jobs=-1)

search = RandomizedSearchCV(
    rf, param_dist, n_iter=10, cv=3, scoring='f1_weighted', n_jobs=-1, random_state=42, verbose=0
)
search.fit(X_train_scaled, y_train_resampled)
best_rf = search.best_estimator_

# 9. Evaluation on test set
y_pred = best_rf.predict(X_test)
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_, output_dict=True)

print("\nClassification Report (Test Set):")
print(json.dumps(report, indent=4))
print(f"Precision: {precision_score(y_test, y_pred, average='weighted'):.4f}")
print(f"Recall: {recall_score(y_test, y_pred, average='weighted'):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred, average='weighted'):.4f}")

# 10. Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix (Test Set)')
plt.savefig('confusion2.png')
plt.close()

# 11. Save outputs
joblib.dump(best_rf, 'model2.pkl')
json.dump(report, open('report2.json', 'w'), indent=4)
np.save('confusion2.npy', cm)
joblib.dump(label_encoder, 'labelencoder2.pkl')
joblib.dump(scaler, 'scaler2.pkl')

print("Saved: model2.pkl, report2.json, confusion2.npy, confusion2.png, labelencoder2.pkl, scaler2.pkl")


Classification Report (Test Set):
{
    "alder": {
        "precision": 0.538235294117647,
        "recall": 0.5683229813664596,
        "f1-score": 0.552870090634441,
        "support": 322.0
    },
    "birch": {
        "precision": 0.46855345911949686,
        "recall": 0.4027027027027027,
        "f1-score": 0.433139534883721,
        "support": 370.0
    },
    "black pine": {
        "precision": 0.6714285714285714,
        "recall": 0.7580645161290323,
        "f1-score": 0.7121212121212119,
        "support": 62.0
    },
    "cherry": {
        "precision": 0.7777777777777778,
        "recall": 0.5675675675675675,
        "f1-score": 0.65625,
        "support": 37.0
    },
    "douglas fir": {
        "precision": 0.8252427184466019,
        "recall": 0.7774390243902439,
        "f1-score": 0.8006279434850864,
        "support": 328.0
    },
    "english oak": {
        "precision": 0.6036866359447005,
        "recall": 0.6223277909738717,
        "f1-score": 0.61286549707602

In [3]:
# 9. Evaluate Model on Final Test Data
test_data_dir = "/kaggle/input/final-test-data"
test_features = []
test_labels = []
test_invalid_samples = []
test_invalid_bands = Counter()
total_samples_attempted = 0  # Track total samples processed

# Check if directory exists and list files
if not os.path.exists(test_data_dir):
    print(f"\nError: Test data directory {test_data_dir} does not exist.")
    exit()
geojson_files = [f for f in os.listdir(test_data_dir) if f.endswith(".geojson")]
print(f"\nFound {len(geojson_files)} GeoJSON files in {test_data_dir}")

# Load all GeoJSON files from test data directory
for file in geojson_files:
    try:
        file_path = os.path.join(test_data_dir, file)
        gdf = gpd.read_file(file_path)
        print(f"Processing file: {file}, Rows: {len(gdf)}")
        total_samples_attempted += len(gdf)  # Count all rows in the file
        for idx, row in gdf.iterrows():
            try:
                patch = []
                for col in band_columns:
                    if col not in gdf.columns:
                        test_invalid_bands[col] += 1
                        array = np.zeros((5, 5), dtype=np.float32)  # Impute with zeros
                        patch.append(array)
                        continue
                    data = row[col]
                    if data is None or (isinstance(data, str) and data.lower() == 'none'):
                        test_invalid_bands[col] += 1
                        array = np.zeros((5, 5), dtype=np.float32)  # Impute with zeros
                        patch.append(array)
                        continue
                    try:
                        parsed_data = ast.literal_eval(data) if isinstance(data, str) else data
                        array = np.array(parsed_data, dtype=np.float32).reshape(5, 5)
                    except (ValueError, SyntaxError, TypeError) as e:
                        test_invalid_bands[col] += 1
                        array = np.zeros((5, 5), dtype=np.float32)  # Impute for parsing errors
                        patch.append(array)
                        continue
                    patch.append(array)
                patch = np.stack(patch, axis=-1)
                if patch.shape != (5, 5, 136):  # Expected shape: 17 bands * 8 months
                    raise ValueError(f"Unexpected patch shape: {patch.shape}")
                test_features.append(patch)
                test_labels.append(row['l3_species'])
            except (ValueError, SyntaxError, TypeError) as e:
                test_invalid_samples.append((file, idx, str(e)))
                continue
    except Exception as e:
        print(f"Failed to process file {file}: {e}")
        continue

# Log invalid samples and bands
print(f"\nTotal samples attempted: {total_samples_attempted}")
print(f"Valid samples processed: {len(test_features)}")
if test_invalid_samples:
    print(f"\nSkipped {len(test_invalid_samples)} invalid test samples:")
    for file, idx, error in test_invalid_samples:
        print(f"File: {file}, Row: {idx}, Error: {error}")
if test_invalid_bands:
    print("\nBands with missing/None/parsing issues in test data:")
    for band, count in test_invalid_bands.most_common():
        print(f"  {band}: {count} times")

# Convert to NumPy arrays
if not test_features:
    print("\nError: No valid test samples loaded. Cannot evaluate model.")
    exit()

X_test_final = np.array(test_features, dtype=np.float32)  # Shape: (N, 5, 5, 136)
y_test_final = np.array(test_labels)

# Preprocess test data
try:
    y_test_final_encoded = label_encoder.transform(y_test_final)  # Use same LabelEncoder
except ValueError as e:
    print(f"Error in label encoding: {e}")
    unknown_labels = set(y_test_final) - set(label_encoder.classes_)
    print(f"Unknown labels in test data: {unknown_labels}")
    exit()
X_test_final_flat = X_test_final.reshape(X_test_final.shape[0], -1)  # Flatten for prediction
X_test_final_scaled = scaler.transform(X_test_final_flat)  # Use same StandardScaler

# Diagnostic: Compare class distributions
print("\nTraining class distribution:")
print(pd.Series(label_encoder.inverse_transform(y_train_resampled)).value_counts())
print("\nTest class distribution:")
print(pd.Series(y_test_final).value_counts())
missing_classes = set(label_encoder.classes_) - set(y_test_final)
print(f"Classes missing in test data: {missing_classes}")

# Evaluate model on test data
y_pred_final = best_rf.predict(X_test_final_scaled)
test_accuracy_final = (y_pred_final == y_test_final_encoded).mean()
print(f"\nFinal Test Data Accuracy: {test_accuracy_final:.4f}")

# Get unique labels in test data to avoid mismatch
unique_test_labels = np.unique(y_test_final_encoded)
unique_test_label_names = label_encoder.inverse_transform(unique_test_labels)

# Additional metrics for test data
report_final = classification_report(
    y_test_final_encoded,
    y_pred_final,
    labels=unique_test_labels,
    target_names=unique_test_label_names,
    output_dict=True
)
print("\nClassification Report for Final Test Data:")
print(json.dumps(report_final, indent=4))
print(f"Recall (Final Test): {recall_score(y_test_final_encoded, y_pred_final, average='weighted'):.4f}")
print(f"F1-Score (Final Test): {f1_score(y_test_final_encoded, y_pred_final, average='weighted'):.4f}")

# Confusion matrix for test data
cm_final = confusion_matrix(y_test_final_encoded, y_pred_final, labels=unique_test_labels)
plt.figure(figsize=(10, 8), dpi=100)
sns.heatmap(cm_final, annot=True, fmt='d', xticklabels=unique_test_label_names, yticklabels=unique_test_label_names, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix for Final Test Data')
plt.savefig('confusion_final.png', dpi=100)
plt.close()

# Total number of test points
print(f"\nTotal Number of Test Points: {len(y_test_final)}")

# Save outputs for test data
json.dump(report_final, open('report_final.json', 'w'), indent=4)
np.save('confusion_final.npy', cm_final)
print("Saved: report_final.json, confusion_final.npy, confusion_final.png")



Found 19 GeoJSON files in /kaggle/input/final-test-data
Processing file: needleleaf_douglas fir_douglas firmar-oct-2022.geojson, Rows: 506
Processing file: broadleaf_short-lived deciduous_aldermar-oct-2022.geojson, Rows: 420
Processing file: broadleaf_beech_european beechmar-oct-2022.geojson, Rows: 1703
Processing file: needleleaf_larch_japanese larchmar-oct-2022.geojson, Rows: 135
Processing file: broadleaf_short-lived deciduous_poplarmar-oct-2022.geojson, Rows: 77
Processing file: needleleaf_pine_scots pinemar-oct-2022.geojson, Rows: 1202
Processing file: broadleaf_oak_english oakmar-oct-2022.geojson, Rows: 645
Processing file: needleleaf_larch_european larchmar-oct-2022.geojson, Rows: 221
Processing file: broadleaf_long-lived deciduous_sycamore maplemar-oct-2022.geojson, Rows: 725
Processing file: broadleaf_long-lived deciduous_european ashmar-oct-2022.geojson, Rows: 432
Processing file: broadleaf_short-lived deciduous_birchmar-oct-2022.geojson, Rows: 353
Processing file: broadleaf

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Total Number of Test Points: 8351
Saved: report_final.json, confusion_final.npy, confusion_final.png
